In [1]:
import torch
from torch import nn, optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [2]:
class Critic(nn.Module):
  def __init__(self, img_channels, features_d, img_size, num_classes):
    super(Critic, self).__init__()
    self.img_size = img_size

    self.disc = nn.Sequential(
        nn.Conv2d(in_channels=img_channels + 1, out_channels=features_d, kernel_size=4, stride=1, padding=1),
        nn.LeakyReLU(0.2),
        self._block(features_d, features_d * 2, 4, 2, 1),
        self._block(features_d * 2, features_d * 4, 4, 2, 1),
        self._block(features_d * 4, features_d * 8, 4, 2, 1),
        nn.Conv2d(features_d * 8, 1, 4, 2, 0)
    )

    self.embedding = nn.Embedding(num_classes, img_size * img_size)

  def _block(self, in_channels, out_channels, kernel_size, stride, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=False),
        nn.InstanceNorm2d(num_features=out_channels, affine=True),
        nn.LeakyReLU(0.2)
    )
  def forward(self, x, labels):
    embeds = self.embedding(labels).view(labels.shape[0], 1, self.img_size, self.img_size)
    x = torch.cat([x, embeds], dim=1)
    return self.disc(x)

In [3]:
class Generator(nn.Module):
  def __init__(self, z_dim, img_channels, features_g, num_classes, img_size, embed_size):
    super(Generator, self).__init__()
    self.img_size = img_size
    self.gen = nn.Sequential(
        self._block(in_channels=z_dim +embed_size,
                    out_channels=features_g * 16,
                    kernel_size=4,
                    stride=1,
                    padding=0),
        self._block(features_g * 16, features_g * 8, 4, 2, 1),
        self._block(features_g * 8, features_g * 4, 4, 2, 1),
        self._block(features_g * 4, features_g * 2, 4, 2, 1),
        nn.ConvTranspose2d(features_g * 2, img_channels, 4, 2, 1),
        nn.Tanh()
    )
    self.embedding = nn.Embedding(num_classes, embed_size)

  def _block(self, in_channels, out_channels, kernel_size, stride, padding):
    return nn.Sequential(
        nn.ConvTranspose2d(in_channels=in_channels,
                           out_channels=out_channels,
                           stride=stride,
                           kernel_size=kernel_size,
                           padding=padding,
                           bias=False),
        nn.InstanceNorm2d(out_channels, affine=True),
        nn.ReLU()
    )

  def forward(self, x, labels):
    embed = self.embedding(labels).unsqueeze(2).unsqueeze(3)
    x = torch.cat([x, embed], dim=1)
    return self.gen(x)

In [4]:
writer_real = SummaryWriter(log_dir='logs/Cond_WGANs/Real')
writer_fake = SummaryWriter(log_dir='logs/Cond_WGANs/Fake')
step = 0

In [5]:
EPOCHS = 5
CRITIC_ITERATIONS = 5
EMBED_SIZE = 100
Z_DIM = 100
LEARNING_RATE = 1e-4
IMG_SIZE = 64
IMG_CHANNELS = 1
NUM_CLASSES = 10
BATCH_SIZE = 64
FEATURES_G = 64
FEATURES_D = 64
LAMBDA_GP = 10
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
def initialize_weights(model):
  for m in model.modules():
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.InstanceNorm2d)):
      nn.init.normal_(m.weight.data, 0.0, 0.2)

In [7]:
def gradient_penalty(critic, real, fake, labels, device='cpu'):
  BATCH_SIZE, C, H, W = real.shape
  epsilon = torch.rand((BATCH_SIZE, 1, 1, 1)).repeat(1, C, H, W).to(device)
  interpolated = real * epsilon + fake * (1 - epsilon)

  mixed_scores = critic(interpolated, labels)

  gradient = torch.autograd.grad(
      inputs=interpolated,
      outputs=mixed_scores,
      grad_outputs=torch.ones_like(mixed_scores),
      create_graph=True,
      retain_graph=True
  )[0]

  gradient = gradient.view(gradient.shape[0], -1)
  gradient_norm = gradient.norm(2, dim=1)
  gradient_penalty = torch.mean((gradient_norm - 1) ** 2)
  return gradient_penalty

In [8]:
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5 for _ in range(IMG_CHANNELS)], [0.5 for _ in range(IMG_CHANNELS)])
])

In [9]:
dataset = datasets.MNIST(root='data/', download=True, transform=transform)

loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [10]:
gen = Generator(Z_DIM, IMG_CHANNELS, FEATURES_G, NUM_CLASSES, IMG_SIZE, EMBED_SIZE).to(device)
initialize_weights(gen)

critic = Critic(IMG_CHANNELS, FEATURES_D, IMG_SIZE, NUM_CLASSES).to(device)
initialize_weights(critic)

In [11]:
gen_opt = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.0, 0.9))
critic_opt = optim.Adam(critic.parameters(), lr=LEARNING_RATE, betas=(0.0, 0.9))

In [12]:
from tqdm import tqdm

critic.train()
gen.train()

for epoch in range(EPOCHS):
  for batch_idx, (real, labels) in enumerate(tqdm(loader)):
    real = real.to(device)
    BATCH_SIZE = real.shape[0]
    labels = labels.to(device)

    for _ in range(CRITIC_ITERATIONS):
      noise = torch.rand((BATCH_SIZE, Z_DIM, 1, 1)).to(device)
      fake = gen(noise, labels)

      critic_fake = critic(fake, labels).reshape(-1)
      critic_real = critic(real, labels).reshape(-1)

      gp = gradient_penalty(critic, real, fake, labels, device=device)

      critic_loss = -(torch.mean(critic_real) + torch.mean(critic_fake)) + (gp * LAMBDA_GP)

      critic.zero_grad()
      critic_loss.backward(retain_graph=True)
      critic_opt.step()

    outputs = critic(fake, labels)
    gen_loss = -torch.mean(outputs)
    gen.zero_grad()
    gen_loss.backward()
    gen_opt.step()

    if batch_idx % 100 == 0 and batch_idx > 0:
      gen.eval()
      critic.eval()
      print(f"\nEpoch [{epoch}/{EPOCHS}], Loss D: {critic_loss:.4f}, loss G: {gen_loss:.4f}")

      with torch.no_grad():
        fake = gen(noise, labels)
        img_grid_real = torchvision.utils.make_grid(real[:32], normalize=True)
        img_grid_fake = torchvision.utils.make_grid(fake[:32], normalize=True)

        writer_real.add_image("Real", img_grid_real, global_step=step)
        writer_fake.add_image("Fake", img_grid_fake, global_step=step)

        step += 1
        gen.train()
        critic.train()

 11%|██▏                 | 101/938 [01:32<13:22,  1.04it/s]


Epoch [0/5], Loss D: -282.9848, loss G: -148.2388


 21%|████▎               | 201/938 [03:03<12:06,  1.02it/s]


Epoch [0/5], Loss D: -739.3724, loss G: -372.5141


 32%|██████▍             | 301/938 [04:34<10:28,  1.01it/s]

...

 85%|█████████████████   | 801/938 [12:12<02:15,  1.01it/s]


Epoch [4/5], Loss D: -143050.4219, loss G: -71531.6406


 96%|███████████████████▏| 901/938 [13:44<00:36,  1.01it/s]


Epoch [4/5], Loss D: -149345.1562, loss G: -74679.5000


100%|████████████████████| 938/938 [14:18<00:00,  1.09it/s]
